In [ ]:
!pip install gradio
!pip install keybert

In [ ]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 
# os.environ["CUDA_VISIBLE_DEVICES"]=""

In [ ]:
# Imports
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline
import numpy as np
import PIL
from PIL import Image
import gradio as gr
import sys
import openai
sys.path.append("../")
import speech_recognition as sr
import pygame
from pygame import mixer
import cv2
from vosk import Model, KaldiRecognizer, SetLogLevel
from pydub import AudioSegment
import wave
import json
from Talking_Head.Talking_Head import Talking_Head
from putting_it_together import blink_loop
%matplotlib inline
import torch
from PIL import Image
import matplotlib.pyplot as plt
import multiprocess
import gradio as gr
import time
import numpy as np
from copy import deepcopy
import asyncio
import threading
import pygame
from pygame import mixer
import os

pygame 2.1.2 (SDL 2.0.18, Python 3.9.15)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
from putting_it_together import WaifuObj

In [ ]:
# Audio recognizer
r = sr.Recognizer()

In [ ]:
# Path to the custom audio model
audio_model_path = "../Audio_Generation/Generation_Scripts/saved_models/default"

# Path to the custom audio data
audio_data_path = "../Audio_Generation/Generation_Scripts/data/albedo"

# Path to the custom model to load in
custom_model_path = "../Finetuning/outputs/r/"

In [ ]:
# The initial summary is initially a basic prompt telling GPT-3 who it is
initial_summ = "You are my female waifu girlfriend who loves me."\
# The initial prompt tells GPT-3 how to respond
initial_prompt = "Me: Hi\nYou: Hello\n\n"\
    "Me: How are you?\nYou: Good. How are you?\n\n"\
    "Me: I'm good.\nYou: Nice to meet you.\n\n"

In [ ]:
# Setup function to setup the environment
# memory_file = "config_file.json"
memory_file = None
Module = WaifuObj(initial_summ, initial_prompt, False, audio_model_path, audio_data_path, custom_model_path, memory_file)

Initializing image model...


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Image model initialized!
Initializing custom text model


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\gabri\AppData\Roaming\Python\Python39\site-packages\transformers\configuration_utils.py │
│ :614 in _get_config_dict                                                                         │
│                                                                                                  │
│   611 │   │   │                                                                                  │
│   612 │   │   │   try:                                                                           │
│   613 │   │   │   │   # Load from local folder or from cache or download from model Hub and ca   │
│ ❱ 614 │   │   │   │   resolved_config_file = cached_file(                                        │
│   615 │   │   │   │   │   pretrained_model_name_or_path,                                         │
│   616 │   │   │   │   │   configuration_file,                                                    │
│   617 │   │   │   │   │   cache_dir=cache_dir,                                                   │
│                                                                                                  │
│ C:\Users\gabri\AppData\Roaming\Python\Python39\site-packages\transformers\utils\hub.py:409 in    │
│ cached_file                                                                                      │
│                                                                                                  │
│    406 │   user_agent = http_user_agent(user_agent)                                              │
│    407 │   try:                                                                                  │
│    408 │   │   # Load from URL or cache if already cached                                        │
│ ❱  409 │   │   resolved_file = hf_hub_download(                                                  │
│    410 │   │   │   path_or_repo_id,                                                              │
│    411 │   │   │   filename,                                                                     │
│    412 │   │   │   subfolder=None if len(subfolder) == 0 else subfolder,                         │
│                                                                                                  │
│ C:\Users\gabri\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\utils\_validators.p │
│ y:114 in _inner_fn                                                                               │
│                                                                                                  │
│   111 │   │   │   kwargs.items(),  # Kwargs values                                               │
│   112 │   │   ):                                                                                 │
│   113 │   │   │   if arg_name == "repo_id":                                                      │
│ ❱ 114 │   │   │   │   validate_repo_id(arg_value)                                                │
│   115 │   │   │                                                                                  │
│   116 │   │   │   elif arg_name == "token" and arg_value is not None:                            │
│   117 │   │   │   │   has_token = True                                                           │
│                                                                                                  │
│ C:\Users\gabri\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\utils\_validators.p │
│ y:172 in validate_repo_id                                                                        │
│                                                                                                  │
│   169 │   │   )                                                                                  │
│   170 │                                                                                          │
│   171 │   if not REPO_ID_REGEX.match(repo_id):             

In [9]:
device = torch.device("cuda:0")

In [ ]:
# Load in the image
# img = Image.open("test.png")
img = Image.open("../Talking_Head/data/illust/../../../test3.png")

In [11]:
# Let's make a global Talking Head class to store the global vector
global img_anim
img_anim = Talking_Head(torch.device("cuda:0"), 0.5, automatic_EMA=True)

# Load in the new image
img_anim.load_new_image(img=deepcopy(img))

# Default pose for the image
img_anim.change_pose()

In [12]:
# Function to extract the word data from a mp3 file
def extract_word_data(filename):
    # Load in the model
    model_path = "../vosk_models/vosk-model-small-en-us-0.15"
    model = Model(model_path)
    
    # Make the audio a wav file
    f = AudioSegment.from_mp3("tmp.mp3")
    f.export("tmp.wav", format="wav")
    
    # Read in the audio
    with wave.open("tmp.wav", "rb") as wf:
        # Prepare the model for rekognition
        rec = KaldiRecognizer(model, wf.getframerate())
        rec.SetWords(True)

        # get the list of JSON dictionaries
        results = []
        # recognize speech using vosk model
        data = wf.readframes(wf.getnframes())
        while len(data) > 0:
            if rec.AcceptWaveform(data):
                part_result = json.loads(rec.Result())
                results.append(part_result)
            data = wf.readframes(wf.getnframes())
        part_result = json.loads(rec.FinalResult())
        results.append(part_result)
    
    audio_trans = results[0]["result"]
    
    # Get the delay between each word
    for i in range(1, len(audio_trans)):
        audio_trans[i]["delay"] = audio_trans[i]["start"]-audio_trans[i-1]["end"]
    audio_trans[0]["delay"] = audio_trans[0]["start"]
    
    return audio_trans

In [13]:
# Literally all this function does is update the
# eye part of the vector every so often
global DONE
DONE = False
async def blink_loop():
    global img_anim
    
    # We want to iterate forever
    while not DONE:
        # Wait a little to blink again
        if img_anim.eye_cycle_end:
            # Blink anywhere between 2 and 7 secods with
            # a mean around 5 seconds (avg blink wait time)
            t = np.clip(np.random.normal(5, 1, size=1)[0], 2, 7)

            # Wait a little before blinking again
            time.sleep(t)
            img_anim.eye_cycle_end = False
        
        # Update the vector
        img_anim.Move_eyes()
        
        # Wait for a new frame to be generated
        while img_anim.eye_frame_disp == False:
            time.sleep(0.001)

# Used to make a thread running the blink loop
def run_blink_loop():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

    loop.run_until_complete(blink_loop())
    loop.close()

In [14]:
async def talk_loop(filename):
    global img_anim
    global generating_mouth_movement
    
    # Get the audio transcript
    audio_trans = extract_word_data(filename)
    
    # Play the audio
    mixer.init()
    mixer.stop()
    mixer.music.unload()
    try:
        mixer.music.load(filename)
        mixer.music.play()
    except pygame.error:
        s = mixer.Sound(filename)
        s.play()
    
    # Iterate over all parts of the audio transcription
    for idx, part in enumerate(audio_trans):
        # Get the beginning and end of the audio piece
        start = part["start"]
        end = part["end"]
        delay = part["delay"]
        
        # Wait for the next audio part according to the
        # delay in the audio. This delay should also take
        # into account the expected generation time of the
        # image as the delay starts after the previous generation
        if idx != 0:
            delay = max(0, delay-img_anim.EMA)
        time.sleep(delay)
        
        # Get the entire audio clip length
        length = end-start
        
        # Setup the mouth movement cycle
        img_anim.setup_mouth_movement(length)
        
        # Mouth movement is being generated
        generating_mouth_movement = True
        
        # Iterate until the movement is done for this part
        while img_anim.mouth_cycle_end == False:
            # Update the vector
            img_anim.Move_mouth()
            
            # Wait for a new frame to be generated
            while img_anim.mouth_frame_disp == False:
                time.sleep(0.001)
                
        # Mouth movement is not being generated
        generating_mouth_movement = False
            
#             # Change the pose and show the image
#             img = img_anim.change_pose()
            
#             # End EMA timer and update EMA
#             if timer_s != -1:
#                 img_anim.update_EMA(time.time()-timer_s)
            
#             # Start timer for EMA
#             timer_s = time.time()
            
#             yield img
    
    
# Used to make a thread running the talk loop
def run_talk_loop(filename):
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

    loop.run_until_complete(talk_loop(filename))
    loop.close()

In [15]:
# Is the mouth movement being generated? This
# is used to inform the event loop if the mouth is
# moving or if just the eyes are moving
global generating_mouth_movement
generating_mouth_movement = False

def event_loop():
    global img_anim
    global generating_mouth_movement
    global add_movement
    global force_gen
    global last_image
    
    # Initial update to make everything visible
    blank_upd = gr.update()
    yield last_image, gr.update(visible=True),\
            gr.update(visible=False)
    
    # Quick calibration. Blink 10 times
    # and calibrate the time it takes
    # to show the image for the EMA
    for i in range(0, 10):
        s = time.time()
        img_anim.eye_cycle_end = False
        while img_anim.eye_cycle_end == False:
            img_anim.Move_eyes()
            img = img_anim.change_pose()
            img_anim.update_EMA(time.time()-s)
            s = time.time()
            yield img, gr.update(), gr.update()
    img_anim.eye_cycle_end = False
    
    # Start the blink loop
    b_thread = threading.Thread(target=run_blink_loop, args=())
    b_thread.start()
    
    while True:
        # If the image is forced to be reloaded, generate
        # the image and reset the flag
        if force_gen == True:
            img = img_anim.change_pose()
            force_gen = False
            yield img, gr.update(), gr.update()
        
        # If movement shouldn't be added, skip the loop
        if add_movement == False:
            time.sleep(0.0001)
            continue
        
        # Wait until a new frame needs to be generated
        if generating_mouth_movement == True:
            if img_anim.mouth_frame_disp == False:
                # Change the pose and show the image
                img = img_anim.change_pose()

                yield img, gr.update(), gr.update()
        else:
            # Start the mouth movement loop
            # m_thread = threading.Thread(target=run_talk_loop, args=("tmp.mp3",))
            # m_thread.start()
            # generating_mouth_movement = True
            if img_anim.eye_frame_disp == False:
                # Change the pose and show the image
                img = img_anim.change_pose()

                yield img, gr.update(), gr.update()
        
        time.sleep(0.0001)

In [ ]:
# io = gr.Interface(event_loop, inputs=[], outputs=[gr.Image().style(height=330)])
# io.queue().launch()

In [ ]:
# Used to start and stop the mouth movement thread from anywhere
global m_thread
m_thread = None

# Used to declare whether movement should be added to the image or not from anywhere
global add_movement
add_movement = True

# Sometimes the image should be forced to be regenerated, this
# flag causes the image to be forced to regenerate once
global force_gen
force_gen = False

# Incase of an error, these can be used. These
# globals are also used to reload stuff
global last_image
global last_text
last_image = np.zeros((50, 50, 3))
last_text = "Error"


def build_img_prompt(text, settings, characteristics):
    # Get the summary and sentiment
    sent = get_sent(text)
    summary = get_summ(text)
    
    # Create the image prompt
    # settings = "1girl, very wide shot, simple background, solo focus, female focus, looking at viewer, ratio:16:9, detailed"
    # characteristics = "waifu, female, brown hair, blue eyes, sidelocks, slight blush, fox ears"
    # sent = "furious"
    # summary = "'I hope get know better' to viewer"
    img_prompt = f"{settings} {characteristics} {','+sent if len(sent)!=0 else ''}, {summary}"
    return img_prompt


def text_to_image(settings, characteristics, guidance_scale, text):
    # Get the image prompt
    img_prompt = Module.build_img_prompt(text, settings, characteristics)
    
    # Get the image
    with Module.suppress_stdout():
        with autocast("cuda"):
            image = Module.imgGen(img_prompt, guidance_scale=guidance_scale)["images"]
            
    
    return image

# Overall function to generate text and audio
def generate_audio(custom_audio, custom_model, text, audio_pth, GPT_key):
    global prompt
    global last_text
    global generating_mouth_movement
    global m_thread
    if m_thread is not None:
        m_thread.join()
    
    # Get the audio if there is any
    if audio_pth:
        # Open the wav file and read in the data
        # Get the audio data
        audio = sr.AudioFile(audio_pth)
        with audio as source:
            audio = r.record(source)
        
        text = audio_to_text(audio)
    
    # Add the text to the current prompt
    Module.cur_prompt += f"Me: {text}\n"
    
    # Get the response
    if custom_model == True:
        ret_text = Module.get_response()
    else:
        try:
            ret_text = ret_text = Module.get_response(GPT_key)
        except:
            gr.Error("GPT key is either invalid or not given")
            return "Error: GPT key is either invalid or not given."
    
    # Create audio and image for the returned text
    if len(ret_text) > 3:
        
        # Create the audio clip
        mixer.stop()
        mixer.music.unload()
        Module.create_audio(ret_text, custom_audio)
        
        # Start the mouth movement loop
        m_thread = threading.Thread(target=run_talk_loop, args=("tmp.mp3",))
        m_thread.start()
        
        # Save the text in case of errors
        last_text = ret_text
        
        return ret_text
    

    
# Function used to generate images
def generate_img(settings, characteristics, guidance_scale):
    global last_image
    global img_anim
    global add_movement
    global force_gen
    
    # Generate an image from the current prompt
    ret_text = ""
    image = text_to_image(settings, characteristics, guidance_scale, ret_text)[0]
    
    # When an image is generated, load it in the animator
    old_add_movement = add_movement
    add_movement = False
    img_anim.load_new_image(img=image)
    add_movement = old_add_movement
    
    # Save the image in case of errors
    last_image = image
    
    # Ensure the image style vector is reset
    img_anim.pose *= 0
    
    # Force the image to be regenerated
    force_gen = True
    

    
    
    
# Transcribes audio to text
def audio_to_text(audio):
    try:
        text = r.recognize_google(audio)
    except sr.UnknownValueError:
        text = "" # Default to nothing
            
    return text


def audio_auto_submit(custom_audio, custom_model, text, audio_pth, GPT_key):
    if audio_pth != None:
        return generate_audio(custom_audio, custom_model, text, audio_pth, GPT_key)
    global last_text
    return last_text

# Initialize the audio mixer
mixer.init()
mixer.music.unload()

# Load in a memory file
def load_mem(filename):
    try:
        Module.load_memory(filename)
        return "Success!"
    except:
        return "Fail! File does not exist or is in incorrect format"
    
# Handle changes to the motion switch which either turns on or
# off image motion
def handle_motion_switch(switch_value):
    global add_movement
    global force_gen
    add_movement = switch_value
    
    # Ensure the image is in the default position
    img_anim.pose *= 0
        
    # Force a reload in the image
    force_gen = True
    
# Handles file uploads
def upload_file(file):
    global last_image
    global img_anim
    global add_movement
    global force_gen
    
    # Load the image as a PIL object
    image = Image.open(file.name)
    
    # When an image is generated, load it in the animator
    old_add_movement = add_movement
    add_movement = False
    img_anim.load_new_image(img=image)
    add_movement = old_add_movement
    
    # Save the image in case of errors
    last_image = image
    
    # Ensure the image style vector is reset
    img_anim.pose *= 0
    
    # Force the image to be regenerated
    force_gen = True
    
    return file.name
    
# Handles image saving
def save_img():
    global last_image
    if not os.path.exists("saved_images"):
        os.mkdir("saved_images")
    filename = fr"./saved_images/{time.ctime().replace(' ', '-').replace(':', '.')}.png"
    if type(last_image) is not PIL.Image.Image:
        Image.fromarray(last_image.clip(0, 255).astype(np.uint8)).save(filename)
    else:
        last_image.save(filename)
        
# Function used to test the mouth movement
def test_mouth():
    global generating_mouth_movement
    global m_thread
    # Make sure the mouth isn't already moving
    if generating_mouth_movement == True:
        return
    
    # Make sure the thread is not running
    if m_thread is not None:
        m_thread.join()
    
    # Start the mouth movement loop
    m_thread = threading.Thread(target=run_talk_loop, args=("test_audio.mp3",))
    m_thread.start()
    

interface = gr.Blocks()
with interface:
    with gr.Tabs():
        with gr.TabItem("Intro"):
            gr.Textbox("""
            Below is an intro explaining how this app works...
            
            Generation Tab:
              Before starting, make sure to click the "Setup interface" button to
              setup the inferface and to begin using the app.
              
              The upper-most part of the interface includes two tabs: 
              "Voice-based Chat" and "Text-based Chat" which are used
              to repond to the AI. Voice-based allows you to use your
              mic to talk to the AI while text-based allows you to
              chat with the AI using text. The audio is auto-submitted
              for response while the text requires either pressing the
              "enter" key or clicking the "Generate Audio" button.
              
              The next part is the "Response" text field. The latest
              response the AI gave will appear here.
              
              Below reponse is a section split into two parts. The left-most
              part is the currently generated image. The rightmost section
              has multiple parts:
              1. "Add motion to image?" checkbox is used to toggle image
                 animation. If checked, the image will be animated. The
                 animation includes blinking and mouth movement assuming
                 the image is in the correct form.
              2. "Mouth movement test" can be used to check if mouth
                 movement works for the current image.
              3. "Save current image" saves the currently generated image
                 to a folder named "saved_images". The filename will
                 be the current time and date so that images don't
                 overwrite eachother
              4. "Upload an image" is used to upload an image you
                 want to load in as opposed to generating one
                 until one looks good. Clikcing on this button allows
                 you to select the image you want to display.
              
              At the bottom of this section, there are two buttons:
              "Generate Audio" and "Generate Image". "Generate Audio"
              takes the currently entered text and generates a new
              response from the AI. "Generate Image" is used to generate
              a new image and display it.
                 
              Some notes about image animation:
                The image must be in the correct form to be animated
                correctly. The image should be a face-shot photo to
                ensure that blinking is done correctly. Mouth movement
                will occur if the image is face forward and when audio
                is generated. Sometimes the mouth movement doesn't
                work and if this is the case, you should probably
                just generate an image until movement works.
            
            
            Settings Tab:
              The settings tab has several uses from loading in past memories
              to changing the style of the image to generate.
              
              The first block in this tab is the "Use custom chat model?" checkbox.
              If this box is checked, a free custom model will be used to
              respond. Otherwise GPT-3 will respond. If the box is unchecked,
              an OpenAI key is required which can obtained following this article:
              https://elephas.app/blog/how-to-create-openai-api-keys-cl5c4f21d281431po7k8fgyol0
              If a key isn't provided, an error will be shown in place of
              the response text.
              
              The next block is the "Settings" blocks which is used to setup
              the style of the image and how it's generated. Settings can be
              found at the following link (though do be warned, the site
              has some sus images, not my doing btw):
              https://danbooru.donmai.us/wiki_pages/tag_group:image_composition
              
              The next block is "Characteristics" which is also used to style
              the generated images. These prompts are more of how you want the
              generated image to look like. Should it be female or male?
              What color hair?
              
              The "settings" block and "characteristics" block actually have no
              difference when implemented, but it's nice to break up the
              difference between image settings and image characteristics.
              
              The next block is the "Guidance value" which is used as a tradeoff
              between Fidelity (how good the image looks) and variance (kind of how
              creative the model is). A value of 1 is required, and having a
              value too high will cause garbage to be produced. Keeping this
              value around 10 seems to work well.
              
              The last part is a memory loading system. As the conversation goes on,
              the conversation is saved to a memory file called "config_file.json". This
              file can be loaded back in through this section of the settings to replace
              the current conversation with a past one saved in a .json file.
              ...
              ...
              ...
              
            """)
        
        with gr.TabItem("Generation"):
            gen_col = gr.Column(visible=False)
            with gen_col:
                # Talking to the AI
                with gr.Tabs():
                    with gr.TabItem("Voice-based Chat"):
                        audio = gr.Audio(source="microphone", type="filepath", label="Response", live=True)
                    with gr.TabItem("Text-based Chat"):
                        text = gr.Textbox(label="Text", value="I love you!", interactive=True)
                response = gr.Textbox(label="Response", value="", interactive=False)

                with gr.Row():
                    # Note gallery expects a 3-D array: (L, W, 3)
                    gallery = gr.Image(label="Generated images", show_label=False)\
                        .style(height=512)

                    with gr.Column():
                        # Switch to generate a new image with audio or keep the
                        # image static
                        motion_switch = gr.Checkbox(value=True, label="Add motion to image?")
                        motion_switch.change(fn=handle_motion_switch, inputs=[motion_switch], outputs=[])

                        # Button to test mouth movement
                        btn_mouth_test = gr.Button("Mouth movement test")
                        btn_mouth_test.click(fn=test_mouth, inputs=[], outputs=[])

                        # Button to save the currently generated image
                        btn_save_img = gr.Button("Save Current Image")
                        btn_save_img.click(fn=save_img, inputs=[], outputs=[])

                        # Button to load an image
                        upload_button = gr.UploadButton("Upload an image", file_types=["image"], file_count="single")
                        upload_button.upload(fn=upload_file, inputs=[upload_button])

                with gr.Row():
                    # Button to generate new audio
                    btn_audio = gr.Button("Generate Audio")

                    # Button to generate new audio
                    btn_img = gr.Button("Generate Image")
            
            # Button to load and setup the generation tab
            btn_load = gr.Button("Setup interface")
            btn_load.click(fn=event_loop, inputs=[], outputs=[gallery, gen_col, btn_load], queue=True)
            
            
        with gr.TabItem("Settings"):
            # Switched for which model to use
            custom_model = gr.Checkbox(value=True, label="Use custom chat model? (False to use GPT, True to use custom model)")
            GPT_key_ = gr.Textbox(label="Key to use GPT-3 (if using GPT-3)\nNote: If you don't have one go here: https://elephas.app/blog/how-to-create-openai-api-keys-cl5c4f21d281431po7k8fgyol0", value="", interactive=True)
            custom_audio = gr.Checkbox(value=False, label="Use custom audio model?")

            # Settings for the image
            settings = gr.Textbox(label="Settings", value= "1girl,solo focus,very wide shot,feamle focus,ratio:16:9,detailed,looking at viewer,facing viewer,facing forward,vtuber", interactive=True)
            characteristics = gr.Textbox(label="Characteristics", value="waifu,female,brown hair,blue eyes,sidelocks,slight blush,happy", interactive=True)
            guidance_scale = gr.Number(label="Guidance value - Tradeoff between creativity and image fidelity (greater than 1.0)", value=10.0, interactive=True, precision=1)
            
            # Used to load a memory file
            with gr.Column():
                mem_file = gr.Textbox(label="Memory File", value= "", interactive=True)
                with gr.Row():
                    mem_load_btn = gr.Button("Load memory file")
                    mem_file_success = gr.Textbox(label="Was the load successful?", value= "", interactive=False)
                    mem_load_btn.click(fn=load_mem, inputs=[mem_file], outputs=[mem_file_success])
            
        # When the audio is changed, we want to auto submit it
        audio.change(fn=audio_auto_submit, inputs=[custom_audio, custom_model, text, audio, GPT_key_], outputs=[response])
        
        # When the button or text is submitted, we want to generate new audio
        btn_audio.click(fn=generate_audio, inputs=[custom_audio, custom_model, text, audio, GPT_key_], outputs=[response])
        text.submit(fn=generate_audio, inputs=[custom_audio, custom_model, text, audio, GPT_key_], outputs=[response])
        
        # When the image button is clicked, we want to generate a new image
        btn_img.click(fn=generate_img, inputs=[settings, characteristics, guidance_scale], outputs=[])

interface.queue(concurrency_count=2).launch(debug=True, share=False)

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\gabri\anaconda3\lib\site-packages\gradio\routes.py", line 337, in run_predict
    output = await app.get_blocks().process_api(
  File "C:\Users\gabri\anaconda3\lib\site-packages\gradio\blocks.py", line 1015, in process_api
    result = await self.call_function(
  File "C:\Users\gabri\anaconda3\lib\site-packages\gradio\blocks.py", line 847, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "C:\Users\gabri\anaconda3\lib\site-packages\anyio\to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *args, cancellable=cancellable,
  File "C:\Users\gabri\anaconda3\lib\site-packages\anyio\_backends\_asyncio.py", line 818, in run_sync_in_worker_thread
    return await future
  File "C:\Users\gabri\anaconda3\lib\site-packages\anyio\_backends\_asyncio.py", line 754, in run
    result = context.run(func, *args)
  File "C:\Users\gabri\anaconda3\lib\site-packages\gradio\utils.py", li